In [36]:
import json
from collections import OrderedDict, defaultdict
from operator import itemgetter

In [5]:
building_name = 'ebu3b'
with open('metadata/{0}_sentence_dict_justseparate.json'.format(building_name), 'r') as fp:
    word_sentence_dict = json.load(fp)
with open('metadata/{0}_label_dict.json'.format(building_name), 'r') as fp:
    word_label_dict = json.load(fp)

In [59]:
# These part could be automated with Brick.
category_dict = {
    'equipment': ['vav', 'chilled_water_pump', 'hot_water_pump', 'ahu', 'supply_fan', 'return_fan', 'exhaust_fan'],
    'network_adapter': ['vnd', 'network_adapter'],
    'idenfitier': ['identifier'],
    'location': ['room', 'floor', 'building', 'server'],
    'none': ['none']
}

In [60]:
def get_key(d, v):
    for k, v_list in d.items():
        if v in v_list:
            return k
    return 'pointtype'

In [63]:
char_sentence_category_dict = dict()
char_sentence_label_dict = dict()
for srcid, word_labels in word_label_dict.items():
    char_sentence_category = list()
    char_sentence_label = list()
    word_sentence = word_sentence_dict[srcid]
    for word, label in zip(word_sentence, word_labels):
        char_sentence_label.append((word[0], 'start_' + label))
        category = get_key(category_dict, label)
        char_sentence_category.append((word[0], 'start_' + category))
        for char in word[1:]:
            char_sentence_category.append((char, 'middle_' + category))
            char_sentence_label.append((char, 'middle_' + label))
    char_sentence_category_dict[srcid] = char_sentence_category
    char_sentence_label_dict[srcid] = char_sentence_label

In [65]:
with open('metadata/{0}_char_category_dict.json'.format(building_name), 'w') as fp:
    json.dump(char_sentence_category_dict, fp, indent=2)
with open('metadata/{0}_char_label_dict.json'.format(building_name), 'w') as fp:
    json.dump(char_sentence_label_dict, fp, indent=2)